In [30]:
import pickle
import numpy as np
import pandas as pd

# Load data

In [31]:
# Load data
admissions = pd.read_csv('mimic_iii_data/ADMISSIONS.csv.gz',compression='gzip').drop('ROW_ID',axis=1)

patients   = pd.read_csv('mimic_iii_data/PATIENTS.csv.gz',compression='gzip').drop('ROW_ID',axis=1)

dx         = pd.read_csv('mimic_iii_data/DIAGNOSES_ICD.csv.gz',compression='gzip').drop('ROW_ID',axis=1)

drg        = pd.read_csv('mimic_iii_data/DRGCODES.csv.gz',compression='gzip').drop('ROW_ID',axis=1)
drg.drop_duplicates(inplace=True)
drg.DESCRIPTION = drg.DESCRIPTION.map(lambda x: x.lower() if pd.notna(x) else x)

In [32]:
maps = {'diagnosis_na'      :'',
        'admission_location':'',
        'admission_na'      :'',
        'ethnicity'         :'',
        'diagnoses'         :'',
        'icd'               :'',
        'drg'               :''}

for n in maps.keys():
    with open(f'{n}_map.pkl', 'rb') as f:
        maps[n] = pickle.load(f)

# Create DF

In [33]:
# Create DF
X = admissions[['HADM_ID','ADMISSION_TYPE','ADMISSION_LOCATION','LANGUAGE','ETHNICITY','DIAGNOSIS']].copy()

### Add Columns

In [34]:
# Lenght of Stay
admissions.DISCHTIME = pd.to_datetime(admissions.DISCHTIME)
admissions.ADMITTIME = pd.to_datetime(admissions.ADMITTIME)
admissions.DEATHTIME = pd.to_datetime(admissions.DEATHTIME)

admissions['LOS'] = (admissions.DISCHTIME - admissions.ADMITTIME).dt.days
# some discharge times are earlier than admission -- all seem to have presented DOA
X['LOS'] = admissions.LOS.map(lambda x: 0 if x<0 else x)
# Add column of pts who died within 4 hours of admission
X['DOA'] = ((admissions.DEATHTIME - admissions.ADMITTIME).dt.total_seconds()/3600<4).astype(int)

In [35]:
# Patient age and gender
T = pd.merge(admissions[['SUBJECT_ID','HADM_ID','ADMITTIME']],
             patients[['SUBJECT_ID','DOB','GENDER']],
             on='SUBJECT_ID')
T_old    = T[T.DOB < '2000-01-01']
T        = T[T.DOB > '2000-01-01']

T['AGE'] =((pd.to_datetime(T.ADMITTIME).dt.date - 
            pd.to_datetime(T.DOB).dt.date).values//(3.1689*10**16)).astype(int)
T_old['AGE'] = 90
T = T.append(T_old)
X = X.merge(T[['HADM_ID','AGE','GENDER']],on='HADM_ID')
X.GENDER = X.GENDER.map(lambda x: 0 if x=='M' else 1)

In [36]:
# ICD9 codes
X = X.merge(dx[['HADM_ID','ICD9_CODE']][dx.SEQ_NUM==1],on='HADM_ID')\
     .rename(columns = {'ICD9_CODE':f'DIAGNOSIS_1'})
    
X = X.merge(dx.groupby('HADM_ID').SEQ_NUM.max(),on='HADM_ID')\
     .rename(columns = {'SEQ_NUM':f'N_DIAGNOSES'})           \
     .set_index('HADM_ID')

# Fill missing values

In [37]:
for col in X.columns:
    print(f'{col}:\n\t{X[col].isna().sum()}')

ADMISSION_TYPE:
	0
ADMISSION_LOCATION:
	0
LANGUAGE:
	25299
ETHNICITY:
	0
DIAGNOSIS:
	15
LOS:
	0
DOA:
	0
AGE:
	0
GENDER:
	0
DIAGNOSIS_1:
	0
N_DIAGNOSES:
	0


### Fill Missing dx

In [38]:
# Impute using missing diagnosis map
X.DIAGNOSIS = X[['DIAGNOSIS','DIAGNOSIS_1']].set_index('DIAGNOSIS_1').DIAGNOSIS\
                                            .fillna(maps['diagnosis_na']).values

### Fill Missing Admisssion Location

In [39]:
# narrow admission categories using admission location map
X.ADMISSION_LOCATION = X.ADMISSION_LOCATION.map(maps['admission_location'])
X.ADMISSION_TYPE     = X.ADMISSION_TYPE.map(lambda x: 'EMERGENCY' if x=='URGENT' else x)

In [40]:
# Check diagnosis for patients missing admission location
X[pd.isna(X.ADMISSION_LOCATION)].DIAGNOSIS.value_counts()

NEWBORN                          300
LIVER LACERATION                   1
ANKLE FRACTURE                     1
CARDIAC ARREST                     1
PNEUMOPERTONEUM                    1
GUN SHOT WOUND                     1
RESPIRATORY FAILURE,UROSEPSIS      1
Name: DIAGNOSIS, dtype: int64

In [41]:
# Check predominant admission location for newboorn dx
X[X.DIAGNOSIS=='NEWBORN'].ADMISSION_LOCATION.value_counts()

REFERRAL    7520
TRANSFER       3
Name: ADMISSION_LOCATION, dtype: int64

In [42]:
# Impute using missing admission location map 
X.ADMISSION_LOCATION = X[['DIAGNOSIS','ADMISSION_LOCATION']].set_index('DIAGNOSIS').ADMISSION_LOCATION\
                                                            .fillna(maps['admission_na']).values

### Fill Missing Language

In [43]:
# missing values assumed to be English. Map to binary, indicating presence of language barrier
X.LANGUAGE = X.LANGUAGE.map(lambda x: 1 if x=='ENGL' else 0) 

In [44]:
# check for any remaining NANs
X.isna().sum().sum()

0

# Narrow categorical
### Broaden ethnicity categories

In [45]:
# narrow ethnicity categories (english vs. non-english speaking)
X.ETHNICITY = X.ETHNICITY.map(maps['ethnicity']) #drop unknown after dummies created

### Map ICD9 codes

In [ ]:
# map ICD9 codes to broader categories using icd map
for regx,diag in maps['icd'].items():
    print(f'Diagnosis:\n\t{diag}')
    dx[f'{diag}'] = dx.ICD9_CODE.str.match(regx,na=False).astype(int)
dx = dx.drop('ICD9_CODE',axis=1).groupby('HADM_ID').sum()

Diagnosis:
	GI_INFECTION
Diagnosis:
	TB
Diagnosis:
	BACTERIAL_INFECTION_OTHER
Diagnosis:
	SEPSIS
Diagnosis:
	HIV
Diagnosis:
	VIRAL_CNS
Diagnosis:
	VIRAL_HEPATITIS
Diagnosis:
	VIRUS_OTHER
Diagnosis:
	CHLAMIDIA
Diagnosis:
	ARTHROPOD_BORN_INFECTION
Diagnosis:
	SYPHILIS
Diagnosis:
	DERMATOPHYTOSIS
Diagnosis:
	CANDIDIASIS
Diagnosis:
	MYCOSES_OTHER
Diagnosis:
	HELMINTHIASIS
Diagnosis:
	INFECTION_OTHER
Diagnosis:
	SARCOIDOSIS
Diagnosis:
	POLIO
Diagnosis:
	ORAL_PHARYNGERAL_CANCER
Diagnosis:
	ESOPHAGUS_STOMACH_CANCER
Diagnosis:
	INTESTINAL_CANCER
Diagnosis:
	LIVER_CANCER
Diagnosis:
	GALLBLADDER_CANCER


In [ ]:
# Remove admission that were present in ICD9 DF but missing from admission DF
missing_patients = set(dx.index) - set(X.index)
dx = dx.drop(missing_patients)
X  = X.join(dx.drop(['SUBJECT_ID','SEQ_NUM'],axis=1))

### Pull Keywords from DRGs

In [ ]:
# Pull tags from DRGs using DRG map
def get_descriptor(s,descriptor):
    return 1 if s[-len(descriptor):]==descriptor else 0
def strip_descriptor(s,descriptor):
    return s[:-len(descriptor)-1] if s[-len(descriptor):]==descriptor else s

tags = ['COMA','SEVERE','VENTILATED','SX','TRANSPLANT','FAILURE','CANCER','ID','NEONATE']

drg_copy = drg.copy()
drg_copy.DESCRIPTION = drg.DESCRIPTION.map(maps['drg'])
for tag in tags:
    drg_copy['TAG_'+tag] = drg_copy.DESCRIPTION.map(lambda x: get_descriptor(x,tag), na_action='ignore')
    drg_copy.DESCRIPTION = drg_copy.DESCRIPTION.map(lambda x: strip_descriptor(x,tag), na_action='ignore')

tags = ['TAG_'+tag for tag in tags]
drg_copy  = drg_copy[tags+['DESCRIPTION','HADM_ID']].drop_duplicates()
drg_copy  = pd.get_dummies(drg_copy,prefix='TAG',columns=['DESCRIPTION'])\
              .groupby('HADM_ID').sum().applymap(lambda x: 1 if x>=1 else 0)
drg_copy.drop('TAG_OTHER',axis=1,inplace=True)
X = X.merge(drg_copy,on='HADM_ID',how='left')
X[drg_copy.columns] = X[drg_copy.columns].fillna(0)

In [ ]:
#tags = ['TAG_COMA','TAG_SEVERE','TAG_VENTILATED','TAG_SX','TAG_FAILURE','TAG_CANCER','TAG_ID','TAG_BREAST', 
#        'TAG_CARDIAC','TAG_CONNECTIVE_TISSUE','TAG_ENDOCRINE','TAG_ENT','TAG_GI','TAG_GU','TAG_HEME','TAG_HIV',
#        'TAG_LIVER','TAG_NEURO','TAG_OB','TAG_ORTHO','TAG_RENAL','TAG_RESPIRATORY','TAG_SKIN_SOFT_TISSUE',
#        'TAG_VASCULAR']
#cols = tags + list(maps['diagnoses'].keys())

In [ ]:
overlap = ['STROKE','TRANSPLANT','NEONATE','DM','ELECTROLYTE_DX','HTN','MI','OD','PSYCH','TRAUMA','SEPSIS']

for d,regx in maps['diagnoses'].items():
    if d in overlap:
        X[f'{d}'] = X.DIAGNOSIS.str.match(regx).fillna(0).astype(int,errors='ignore')
    else:
        X[f'TAG_{d}'] = X.DIAGNOSIS.str.match(regx).fillna(0).astype(int,errors='ignore')
        
# Merge overlapping columns
for d in overlap:
    X[f'TAG_{d}'] = X[[d,'TAG_'+d]].apply(lambda x: 1 if sum(x)>0 else 0,axis=1)
    X.drop(d,axis=1,inplace=True)

### Drop any 1-level columns

In [ ]:
one_level_columns = [c for c in X.columns if len(X[c].unique())<2]
print(one_level_columns)
X = X.drop(one_level_columns,axis=1)

In [ ]:
# Last check for missing values
X.isna().sum().sum()

# Save

In [ ]:
X = X.drop(['DIAGNOSIS','DIAGNOSIS_1','DOA'],axis=1)

In [ ]:
X.to_csv('X_y_IV.csv')